**A) Datahantering och Beskrivande statistik**
*Vald indikator (N07900) Tillgång till bredband om minst 100 Mbit/s, andel (%)*

Andel av kommunens hushåll som har tillgång till bredband om minst 100 megabit per sekund. PTS (Post- och telestyrelsen) begär in underlag om täckning för accesstekniker av berörda parter, underlaget matchas sedan med Fastighetsregistret. Det insamlade underlaget ihop med ett antal antaganden utgör grunden för kartläggningen. För tillgång till riktigt höga hastigheter, 100 Mbit/s under gynnsamma omständigheter och minst 50 Mbit/s i bråd timme, fordras idag trådbundna accesstekniker i form av fiber- eller kabel-tv-nät. Avser situationen
1 oktober. T.o.m. 2014 avsågs befolkning istället för hushåll. Källa: Post- och telestyrelsen, PTS Bredbandskartläggning.


In [10]:
%pip install plotly;
import plotly.express as px
import pandas as pd

# Ange sökvägen till CSV-filen
csv_path = r"C:\Users\fabia\AH1023\Projekt\Projekt_1\pt_data.csv"

# Läs in datan från CSV-filen
df = pd.read_csv(csv_path)

col = "N07900"
s = pd.to_numeric(df[col], errors="coerce").dropna()
# ange/beräkna för den valda indikatorn: min, max, medel, median och standardavvikelse. 
print(f"{col} — count: {s.count()}")
print(f"min:    {s.min()}")
print(f"max:    {s.max()}")
print(f"mean:   {s.mean()}")
print(f"median: {s.median()}")
print(f"std:    {s.std()}")
# plotta histogram utifrån vald variabel 
fig = px.histogram(df, x="N07900", nbins=30, title="Histogram of N07900")
fig.show()

Note: you may need to restart the kernel to use updated packages.
N07900 — count: 290
min:    0.18836826
max:    96.01088065
mean:   35.08771286182758
median: 30.78244899
std:    21.884208603019868


Beskrivning och resonemang  av histogram 

**B) Regressionsanalys**
Låt variabeln pt_share (andelen i procent av resorna som sker med kollektivtrafik) vara svarsvariabeln (Y). Välj ut åtta indikatorer du tror att Y kan bero på. Ta även med en dummyvariabel, till exempel kommungrupp. Se länken ovan om Categorical variables. För att välja ut ett subset av kolumnerna i en pandas-tabell använd filter (kolumnnamnen är bara exempel)


In [ ]:
#gör ett subset av datframen med de valda variablerna 

df_subset = df.filter(["pt_share", "U85001", "U85421", "N00914", "N07900", "U1803", "N02937", "U85002","N00905"], axis=1)

#Gör en scatterplot där du plottar Y mot förklaringsvariabeln du valde i A. 
fig = px.scatter(df, x="N07900", y="pt_share", title="Scatter plot pt_share vs N07900")
fig.show()
# Gör även en scatterplot-matris för parvis analys av beroendet mellan förklaringsvariablerna. Analysera hur beroendet ser ut.
fig = px.scatter_matrix(df_subset, dimensions=["pt_share", "U85001"])
fig.show() 
# Funktionen scatter_matrix sköter plottandet:


fig = px.scatter_matrix(df_subset, dimensions=["pt_share", "U85001", "U85421", "N00914", "N07900", "U1803", "N02937", "U85002","N00905"])
fig.show()



ValueError: Value of 'dimensions_5' is not the name of a column in 'data_frame'. Expected one of ['pt_share', 'U85001', 'U85421', 'N00914', 'N07900', 'N02937', 'U85002', 'N00905'] but received: U1803

**C) Validering**
Dela in data i två delar mha funktionerna sample() och drop() som finns i Pandas. Dvs ifall ni har data i en DataFrame som heter df så kan ni använda

In [ ]:
estimation_data = df.sample(frac=0.75)

validation_data = df.drop(estimation_data.index)
#Skatta er bästa modellkonfiguration från B) på tre fjärdedelar av datamaterialet och använd sedan den sista fjärdedelen till att validera er modell.

# Plotta modellberäknat mot observerat i en scatterplot. 

#För att prediktera på valideringsdatat på er modell från B som här i stället har tränats på estimeringsdatat kan man skriva denna koden:
pt_share_predicted = ditt_valda_modell_namn.predict(validation_data[["din_första_indikator", "din_andra_indikator"...]]) 
#Exemplet innehåller bara två indikatorer men går att köras med fler. 
# För att sedan göra resultatet till en tabell, döpa om kollumnnamnet samt lägga till denna till validation_data tabellen kan man köra:
pt_share_predicted = pt_share_predicted.to_frame()

pt_share_predicted.columns = ['pt_share_predicted']

observed_plus_predicted = validation_data.join(pt_share_predicted)